# More `astroplan` - Constraints

In [ ]:
import numpy as np

import astropy.units as u
from astropy.table import QTable
from astropy.time import Time
from astropy.coordinates import SkyCoord

from astroplan import Observer, FixedTarget

import pytz

import warnings
warnings.filterwarnings('ignore', category=Warning)

----

# Side Topic One: `For-loop` and `arrays`

In [ ]:
my_matrix = np.array(
[[0, 3, 7],
 [2, 1, 8],
 [4, 9, 3],]
)

my_matrix

In [ ]:
for my_x, my_y, my_z in my_matrix:
    print(f"X = {my_x}, Y = {my_y}, and Z = {my_z}")

---
# Side Topic Two: List Comprehensions

<img src="https://uwashington-astro300.github.io/A300_images/ComplexCode.jpg" width="275"/>

List comprehensions provide a concise way to create lists (arrays). Common applications are to make new lists where each element is the result of some operations applied to each member of another sequence.

### For example: Create the list: `[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]`

In [ ]:
squares = []                    # create a blank list

for x in range(10):             # for loop 0 -> 9
    squares.append(x**2)        # calculate x**2 for each x, add to end of list

squares

### You can do the same thing with:

In [ ]:
squares = [x**2 for x in range(10)]

squares

### You can include `if` statements:

In [ ]:
even_squares = []

for x in range(10):

    if (x % 2 == 0):
        even_squares.append(x**2)

even_squares

### You can do the same thing with:

In [ ]:
even_squares = [x**2 for x in range(10) if (x % 2 == 0)]

even_squares

----
# Side Topic Three: 

* ## `For-loop - arrays`
* ## List Comprehensions


In [ ]:
my_matrix

In [ ]:
def my_function(my_x, my_y, my_z):
    result = f"X = {my_x}, Y = {my_y}, and Z = {my_z}"
    return result

In [ ]:
my_output = [my_function(my_x, my_y, my_z) for my_x, my_y, my_z in my_matrix]

In [ ]:
my_output

---

# Now to observations

## Start with a external list of target objects:

In [ ]:
target_table = QTable.read('https://uwashington-astro300.github.io/A300_Data/ObjectList.csv', 
                           format='ascii.csv')

In [ ]:
target_table[0:3]

## Add some units to the table

In [ ]:
target_table['RA'].unit = u.hourangle
target_table['DEC'].unit = u.deg

In [ ]:
target_table[0:3]

In [ ]:
for my_Name, my_RA, my_DEC in target_table:
    print (my_Name, my_RA, my_DEC)

### Instead of using SIMBAD, you can manually enter info for a target in the sky

`FixedTarget(coords = SkyCoord(RA, DEC), name = 'NAME')`

In [ ]:
my_targets = [FixedTarget(coord = SkyCoord(ra = my_RA, dec = my_DEC), name = my_Name)
           for my_Name, my_RA, my_DEC in target_table]

In [ ]:
my_targets

----

# [Theodor Jacobsen Observatory (TJO)](http://depts.washington.edu/astron/outreach/jacobsen-observatory/)

The UW Astronomy department was founded in 1891 by the mathematics professor Joseph M. Taylor. Dr. Taylor built the first observatory on the University of Washington campus (UW was originally located downtown - 4th Ave and University, the current location of the Fairmont Olympic Hotel).
When the university moved to its present location, Dr. Taylor spent \$3,000  on a 6-inch refractor telescope and appropriated the stone and money left over from the construction of Denny Hall to build the observatory we see on campus today.

The observatory is named after Theodor S. Jacobsen, who began teaching at UW in 1928. Professor Jacobsen had a long career at the UW and was the key figure in establishing the modern UW Astronomy department. His last book came out in 1999, four years before his death at the age of 102.

<p align="center"> 
    <img src="https://uwashington-astro300.github.io/A300_images/TJO_Now_sm.jpg" width = "291">
    <img src="https://uwashington-astro300.github.io/A300_images/TJO_Telescope.jpg" width = "176">
    <img src="https://uwashington-astro300.github.io/A300_images/TJ_Photo.jpg" width = "243">
    <img src="https://uwashington-astro300.github.io/A300_images/OldUW.jpg" width = "320">
</p>

In [ ]:
tjo_observe = Observer(longitude = -122.309268 * u.deg,
                       latitude = 47.660431 * u.deg,
                       timezone = 'US/Pacific',
                       name = "Theodor Jacobsen Observatory"
                      )

In [ ]:
tjo_observe

----

# Public Outreach

TJO's main use today is as a site for public outreach. In normal times, undergraduate astro majors (you!) give monthly public talks and lead tours of the night sky

For this example, let us say you are planning to give a talk at TJO about the constellations of the zodiac. And you want to be able to show the audience some of the constellations in the sky.

* The date of the talk is Friday Apr 7, 2023
* The talks are usually given from 8 - 11 pm [PT]

In [ ]:
talk_start = Time("2023-04-08 04:00")
talk_end = Time("2023-04-08 07:00")

print(talk_start.to_datetime(tjo_observe.timezone))

In [ ]:
my_time_range = [talk_start, talk_end]

my_time_range

----

# Observing Constraints

### We have a place, a time, and some targets. But what targets can we see?

In [ ]:
from astroplan import SunSeparationConstraint
from astroplan import observability_table

### We are observing at night, so let us only look at things thar are greater than 6 hours away from the Sun

In [ ]:
my_constraints = [SunSeparationConstraint(6 * u.hourangle)]

In [ ]:
my_constraints

## astroplan's `observability_table` is a quick summary of what you can see

`observability_table(constraints, location, targets, time_range)`

In [ ]:
observing_table = observability_table(my_constraints, tjo_observe, my_targets, time_range = my_time_range)

observing_table

### Just show the objects that are observable

In [ ]:
observing_table[observing_table['ever observable'] == True]

## What about Moonlight?

In [ ]:
tjo_observe.moon_illumination(talk_start)

In [ ]:
from astroplan import MoonSeparationConstraint

In [ ]:
my_constraints.append(MoonSeparationConstraint(30 * u.deg))

In [ ]:
my_constraints

In [ ]:
observing_table = observability_table(my_constraints, tjo_observe, my_targets, time_range = my_time_range)

In [ ]:
observing_table[observing_table['ever observable'] == True]

---
# Site Specific Constraints

<img src="https://uwashington-astro300.github.io/A300_images/TJO_Locate.PNG" width="300"/>

## TJO is not exactly in the greatest location for astronomical observations!

* Lots of city lights
* Especially the adjacent parking lot
* Surrounding trees really limit the view!
* You have to look above Alt = 30 degrees to clear the trees

In [ ]:
from astroplan import AltitudeConstraint

In [ ]:
my_constraints.append(AltitudeConstraint(min=30*u.deg, max=None))

In [ ]:
observing_table = observability_table(my_constraints, tjo_observe, my_targets, time_range = my_time_range)

In [ ]:
observing_table[observing_table['ever observable'] == True]

In [ ]:
total_time = (talk_end - talk_start).to(u.h)

total_time

In [ ]:
observing_table['Observing Time'] = total_time * observing_table['fraction of time observable']

In [ ]:
observing_table[observing_table['ever observable'] == True]

----
# Non-Standard Constraint

<img src="https://uwashington-astro300.github.io/A300_images/TJO_Locate_AZ.PNG" width="300"/>

## In addition, the trees to the North and West completely envelop the observatory!

* You can only really observe at Azimuths between about 70 and 185 degrees
* `astroplan` does not have an Azimuth constraint so will will try something else

## We will use `plot_sky` to check the azimuth constraint

* `plot_sky(targets, location, time)`

In [ ]:
target_mask = np.flatnonzero(observing_table['ever observable'] == True)

In [ ]:
target_mask

In [ ]:
for index in target_mask:
    print(my_targets[index])

In [ ]:
import matplotlib.pyplot as plt

from astroplan import time_grid_from_range
from astroplan.plots import plot_sky

In [ ]:
fig, ax = plt.subplots(
    subplot_kw={'projection': 'polar'},
    figsize = (8, 8), 
    constrained_layout = True
)

for index in target_mask:
    ax = plot_sky(my_targets[index], tjo_observe, talk_start)

ax.legend(loc=0,shadow=True);

In [ ]:
fig, ax = plt.subplots(
    subplot_kw={'projection': 'polar'},
    figsize = (8, 8), 
    constrained_layout = True
)

for index in target_mask:
    ax = plot_sky(my_targets[index], tjo_observe, talk_end)

ax.legend(loc=0,shadow=True);

## Looks like we are going to talk about Leo!

----

# Astroplan Constraints

`from astroplan import CONSTRAINT`

* `AirmassConstraint(max)` - Constrain the airmass of a target.
* `AltitudeConstraint(min, max)` - Constrain the altitude of the target.
* `MoonIlluminationConstraint(min, max)` - Constrain the fractional illumination of the Moon.
* `MoonSeparationConstraint(min, max)` - Constrain the separation between the Moon and some targets.
* `SunSeparationConstraint(min, max)` - Constrain the separation between the Sun and some targets.

---

# Moving Targets (solar system targets)

- The built-in solar system objects are: 'sun', 'mercury', 'venus', 'earth-moon-barycenter', 'earth', 'moon', 'mars', 'jupiter', 'saturn', 'uranus', 'neptune', 'pluto'

In [ ]:
from astropy.coordinates import get_body

In [ ]:
sun_now = get_body('sun',talk_start)

sun_now

In [ ]:
sun_location = tjo_observe.altaz(talk_start, sun_now)

sun_location.alt, sun_location.az

In [ ]:
moon_now = get_body('moon',talk_start)

In [ ]:
moon_location = tjo_observe.altaz(talk_start, moon_now)

moon_location.alt, moon_location.az

In [ ]:
sun_now.separation(moon_now)

## We can make the Moon a `FixedTarget` for planning

In [ ]:
pseudo_moon = FixedTarget(moon_now, name = "Moon")

In [ ]:
pseudo_moon.coord

In [ ]:
fig, ax = plt.subplots(
    subplot_kw={'projection': 'polar'},
    figsize = (8, 8), 
    constrained_layout = True
)

ax = plot_sky(pseudo_moon, tjo_observe, talk_start + 1 * u.h)
ax = plot_sky(pseudo_moon, tjo_observe, talk_end)

ax.legend(loc=0,shadow=True);

## Maybe we should talk about the Moon!